In [1]:
import pathlib
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
# from model import Net
import matplotlib.pyplot as plt

from torch import nn
import tqdm


In [2]:
# Create a pytorch dataset
data_dir = pathlib.Path('/data/tiny-imagenet/')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
print('Discovered {} images'.format(image_count))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 15

# data_transforms = transforms.Compose([
#     transforms.ToTensor(),
#     # transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
# ])

data_transforms = transforms.Compose([
    transforms.RandomCrop(64, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create a simple model

model = torchvision.models.resnet18(pretrained=True)
for param in list(model.parameters())[:-10]:
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features

model.fc =  nn.Sequential(
                  nn.Linear(num_ftrs, 256), 
                  nn.ReLU(), 
                  nn.Linear(256, 256), 
                  nn.ReLU(), 
                  nn.Linear(256, 200))
model = model.to(device)
# model = Net(len(CLASS_NAMES), im_height, im_width)
optim = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
print(len(train_loader))
for i in range(num_epochs):
    train_total, train_correct = 0,0
    for idx, (inputs, targets) in enumerate(train_loader):
        # print(inputs[1])
        # plt.imshow(inputs[1].permute(1,2,0))
        # plt.show()
        inputs, targets = inputs.to(device), targets.to(device)
        optim.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optim.step()
        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()
        # if idx % 100 == 0:
        print("\r", end='')
        print(f'training {100 * idx / len(train_loader):.2f}%: {train_correct / train_total:.3f}', end='')
    torch.save({
        'net': model.state_dict(),
    }, 'resnet50-5epochs-noaug.pt')


##test #1:0.038 at 5.06%


Discovered 120000 images
3125
training 99.97%: 0.180

FileNotFoundError: [Errno 2] No such file or directory: './results/resnet50-5epochs-noaug.pt'

In [26]:
validation_set = torchvision.datasets.ImageFolder(data_dir / 'val', data_transforms)
val_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)

model.eval()
all_preds = []
all_labels = []
all_losses = []
with torch.no_grad():
    index = 0
    for batch in tqdm.tqdm(val_loader):
        inputs = batch[0]
        targets = batch[1]
        targets = targets.cuda()
        inputs = inputs.cuda()
        preds = model(inputs)
        loss = nn.CrossEntropyLoss()(preds, targets)
        all_losses.append(loss.cpu())
        all_preds.append(preds.cpu())
        all_labels.append(targets.cpu())

100%|██████████| 313/313 [00:02<00:00, 148.98it/s]


## Test Particular Model
Run to load particular model onto computer

In [ ]:
model = torch.load('')

## Top 1 Accuracy

In [27]:
top_preds = [x.argsort(dim=-1)[:,-1:].squeeze() for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds).sum()
accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 1 Validation Accuracy: {accuracy}")

Top 1 Validation Accuracy: 0.3907747603833866


## Top 3 Accuracy

In [28]:
top_preds = [x.argsort(dim=-1)[:,-3:] for x in all_preds]
correct = 0
for idx, batch_preds in enumerate(top_preds):
    correct += torch.eq(all_labels[idx], batch_preds[:,0:1].squeeze()).sum()
    correct += torch.eq(all_labels[idx], batch_preds[:,1:2].squeeze()).sum()

    correct += torch.eq(all_labels[idx], batch_preds[:,2:3].squeeze()).sum()

accuracy = correct.item() / (32 * len(all_labels))
print(f"Top 3 Validation Accuracy: {accuracy}")

Top 3 Validation Accuracy: 0.5749800319488818
